In [ ]:
import os, sys
import numpy as np
import torch
import json
from scipy.special import softmax

sys.path.append("..")
from singleVis.SingleVisualizationModel import SingleVisualizationModel
from singleVis.data import DenseActiveLearningDataProvider
from singleVis.projector import DenseALProjector

In [ ]:
CONTENT_PATH = "/home/xianglin/DVI_data/active_learning/random/resnet18/CIFAR10"
GPU_ID = "1"
epoch_num = 200
iteration = 1

In [ ]:
sys.path.append(CONTENT_PATH)
from config import config

In [ ]:
SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
BASE_ITERATION =config["BASE_ITERATION"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
LAMBDA = VISUALIZATION_PARAMETER["LAMBDA"]
S_LAMBDA = VISUALIZATION_PARAMETER["S_LAMBDA"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
INIT_NUM = VISUALIZATION_PARAMETER["INIT_NUM"]
ALPHA = VISUALIZATION_PARAMETER["ALPHA"]
BETA = VISUALIZATION_PARAMETER["BETA"]
MAX_HAUSDORFF = VISUALIZATION_PARAMETER["MAX_HAUSDORFF"]
HIDDEN_LAYER = VISUALIZATION_PARAMETER["HIDDEN_LAYER"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
T_N_EPOCHS = VISUALIZATION_PARAMETER["T_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]

# define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
model = SingleVisualizationModel(input_dims=512, output_dims=2, units=256, hidden_layer=HIDDEN_LAYER)

import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))
data_provider = DenseActiveLearningDataProvider(CONTENT_PATH, net, BASE_ITERATION, epoch_num, split=-1, device=DEVICE, classes=CLASSES,verbose=1)
projector = DenseALProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name="al_hybrid", device=DEVICE)


In [ ]:
from singleVis.visualizer import DenseALvisualizer
vis = DenseALvisualizer(data_provider, projector, 300)

In [ ]:
epoch = 200
data = data_provider.train_representation_lb(iteration, epoch)
labels = data_provider.train_labels_lb(iteration)
pred = data_provider.get_pred(iteration, epoch, data).argmax(1)
vis.savefig_cus(iteration, epoch, data, pred, labels)
# vis.savefig(1,200)

In [ ]:
samples = data_provider.train_representation(iteration, epoch_num)
pred = data_provider.get_pred(iteration, epoch_num, samples)
confidence = np.amax(softmax(pred, axis=1), axis=1)
uncertainty = 1-confidence

In [ ]:
samples = np.zeros((200, 50000, 512))
for i in range(1, 201, 1):
    samples[i-1] = data_provider.train_representation(iteration, i)

In [ ]:
embeddings_2d = np.zeros((200, 50000, 2))
for e in range(1, 201, 1):
    embeddings_2d[e-1] = projector.batch_project(iteration, e, samples[e-1])
embeddings_2d = np.transpose(embeddings_2d, [1,0,2])
labels = data_provider.train_labels(iteration)

In [ ]:
path = os.path.join(CONTENT_PATH, "Model", "Iteration_{}".format(iteration),"embeddings.npy")
embeddings_2d = np.load(path)

In [ ]:
path = os.path.join(CONTENT_PATH, "Model", "Iteration_{}".format(iteration),"embeddings.npy")
np.save(path,embeddings_2d)

In [ ]:
# embeddings_2d = np.transpose(embeddings_2d, [1,0,2])
samples.shape,embeddings_2d.shape

In [ ]:
from singleVis.trajectory_manager import TrajectoryManager, FeedbackTrajectoryManager, Recommender
# tm = FeedbackTrajectoryManager(samples, embeddings_2d,30, period=80,metric="a")
# tm = TrajectoryManager(samples, embeddings_2d, 30, period=80,metric="a")
tm = Recommender(uncertainty, embeddings_2d, 30, period=80,metric="a")

In [ ]:
tm.clustered()

In [ ]:
data = data_provider.train_representation(iteration, epoch_num)
labels = data_provider.train_labels(iteration)
pred = data_provider.get_pred(iteration, epoch_num, data).argmax(1)
wrong_pred_idx = np.argwhere(pred!=labels).squeeze()
wrong_pred_idx.shape

In [ ]:
lb_idx = data_provider.get_labeled_idx(iteration)
ulb_idxs = data_provider.get_unlabeled_idx(50000,lb_idx)

In [ ]:
ignore_rate = 0.1
remain_rate = 1-ignore_rate

In [ ]:
selected = tm.sample_batch(100)
remain_selected = np.random.choice(selected, size=int(len(selected)*remain_rate))
valid_selected = np.intersect1d(remain_selected, ulb_idxs)
correct = np.intersect1d(valid_selected, wrong_pred_idx)
wrong = np.setdiff1d(valid_selected, correct)
print(len(correct), len(remain_selected))

In [ ]:
# just sampling
for _ in range(10):
    tm.update_belief(selected)
    selected = tm.sample_batch(100)
    remain_selected = np.random.choice(selected, size=int(len(selected)*remain_rate))
    valid_selected = np.intersect1d(remain_selected, ulb_idxs)
    correct = np.intersect1d(valid_selected, wrong_pred_idx)
    wrong = np.setdiff1d(valid_selected, correct)
    print(len(correct), len(remain_selected))

In [ ]:
selected = tm.sample_batch(100)
correct = np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)
wrong = np.setdiff1d(selected, correct)
print(len(correct), 100)

In [ ]:
# success rate
for _ in range(10):
    tm.update_belief(correct, wrong)
    selected = tm.sample_batch(100)
    correct = np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)
    wrong = np.setdiff1d(selected, correct)
    print(len(correct), 100)

In [ ]:
s = np.random.choice(ulb_idxs,size=200,replace=False)

In [ ]:
# random
s = np.random.choice(ulb_idxs,size=200,replace=False)
len(np.intersect1d(s, wrong_pred_idx)), len(s)

In [ ]:
selected, _ = tm.sample_batch_init(10)
correct = np.intersect1d(selected, wrong_pred_idx)
wrong = np.setdiff1d(selected, correct)
print(len(correct), 10)

In [ ]:
# kernel regression
for _ in range(10):
    selected,_ = tm.sample_batch(correct, wrong, 10)
    c = np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)
    w = np.setdiff1d(selected, correct)
    print(len(c), 10)
    correct = np.concatenate((correct, c), axis=0)
    wrong = np.concatenate((wrong, w), axis=0)